In [5]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from datetime import datetime 

import time

In [6]:
#  parameters
circle="Zone"
month="nov2023"
month_doc=9
month_unpaid=12
date='28122023'
ftpHost = 'ftp.uppclonline.com'
ftpUname = 'mvftpreport'
ftpPass = 'Mvftp@321'
localFolderPath = f".\\{month}\\billed"
remoteFolder = f"CSV_BILLED/{date}"

In [7]:
import os, shutil
folder = f".\\{month}\\billed"
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))
        
folder = f".\\{month}\\unbilled"
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [8]:
import ftplib
import os
import gzip
import shutil
import time



def downloadFilesFromFtp(localfolderPath, targetFilenames, ftpHost, ftpUname, ftpPass, remoteWorkingDirectory):
    # initialize the flag that specifies if download is success
    isDownloadSuccess: bool = False

    # create an FTP client instance, use the timeout parameter for slow connections only
    ftp = ftplib.FTP(timeout=30)
    ftp.encoding = "utf-8"

    # connect to the FTP server
    ftp.connect(ftpHost)

    # login to the FTP server
    ftp.login(ftpUname, ftpPass)

    # change current working directory if specified
    if not (remoteWorkingDirectory == None or remoteWorkingDirectory.strip() == ""):
        _ = ftp.cwd(remoteWorkingDirectory)

    # iterate through each remote filename and download
    for fItr in range(len(targetFilenames)):
        targetFilename = targetFilenames[fItr]
        print(targetFilename)
        # derive the local file path by appending the local folder path with remote filename
        localFilePath = os.path.join(localfolderPath, targetFilename)
        print(localFilePath)
        print("downloading file {0}".format(targetFilename))
        # download FTP file using retrbinary function
        with open(localFilePath, "wb") as file:
            retCode = ftp.retrbinary("RETR " + targetFilename, file.write)
         
        with gzip.open(localFilePath, 'rb') as f_in:
            with open(localFilePath[:-3], 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        os.remove(localFilePath)
    # read file
            
    # send QUIT command to the FTP server and close the connection
    ftp.quit()

    # check if download is success using the return code (retCode)
    if retCode.startswith('226'):
        isDownloadSuccess = True
    return isDownloadSuccess



remoteFilenames = [ f'BILLED_MVVNL_DIV354921_{date}.csv.gz', f'BILLED_MVVNL_DIV354922_{date}.csv.gz', f'BILLED_MVVNL_DIV354923_{date}.csv.gz', f'BILLED_MVVNL_DIV355131_{date}.csv.gz', f'BILLED_MVVNL_DIV355132_{date}.csv.gz', f'BILLED_MVVNL_DIV355133_{date}.csv.gz', f'BILLED_MVVNL_DIV355211_{date}.csv.gz', f'BILLED_MVVNL_DIV355212_{date}.csv.gz', f'BILLED_MVVNL_DIV355213_{date}.csv.gz', f'BILLED_MVVNL_DIV355214_{date}.csv.gz']


# run the function to download the files from FTP server
isDownloadSuccess = downloadFilesFromFtp(
    localFolderPath,remoteFilenames, ftpHost,ftpUname, ftpPass, remoteFolder)

print("download status = {0}".format(isDownloadSuccess))

BILLED_MVVNL_DIV354921_28122023.csv.gz
.\nov2023\billed\BILLED_MVVNL_DIV354921_28122023.csv.gz
downloading file BILLED_MVVNL_DIV354921_28122023.csv.gz


error_perm: 550 Failed to open file.

In [ ]:
bill="billed"

In [ ]:
def loop(file_name):
    return pd.read_csv(f"{month}/{bill}/{file_name}")

In [ ]:
mylist = os.listdir(f"{month}/{bill}")

In [ ]:
mylist

In [ ]:
start = time.time()
billed = []
billed = Parallel(n_jobs=-1, verbose=10,prefer="threads")(delayed(loop)(file_name) for file_name in mylist)
billed = pd.concat(billed, ignore_index=True)

end = time.time()
print("CSV:", end - start)

In [ ]:
billed = billed[billed['ACCT_ID'].notna()]

In [ ]:
cat = [ 'LMV2', 'LMV6','LMV4']

In [ ]:
div_billed=billed[(billed.TARIFF_TYPE.isin(cat))].pivot_table(index=['CIRCLE_NAME','DIV_NAME'], columns=['BILL_BASIS'],values=['ACCT_ID'],
                    aggfunc={'ACCT_ID':'count'},fill_value=0 )

In [ ]:
div_billed.to_excel(f"{month}/zone/report/{circle}_{date}_billed.xlsx")